In [ ]:
import os
import glob
import requests


def localize_images(markdown_file, output_dir):

    with open(markdown_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    id = 0
    new_lines = []
    for line in lines:
        if "](https://cdn.nlark.com" in line:
            print(line)
            start_index = line.index("](") + 2
            end_index = len(line)-2
        
            img_url =  line[start_index:end_index]
            img_name = img_url.split("#")[0].split("/")[-1]
            img_filename = f'{markdown_file.split("/")[1].split(" ")[0]}-{id}.{img_name.split(".")[-1]}'

            response = requests.get(img_url)
            with open(f'assets/{img_filename}', 'wb') as f:
                f.write(response.content)
            
            line = line.replace(img_url,f"../assets/{img_filename}")
            line = "\n"+"![" + line[line.index("]"):]+"\n"
            id+=1
        
            new_lines.append(line)

            
        else:
            new_lines.append(line)


    # 写入替换好的 Markdown 文件
    replaced_file = markdown_file.replace("yuque","pages")
    
    replaced_file = replaced_file.split()[0] + ".md"
    with open(replaced_file, 'w', encoding='utf-8') as f:
        markdown_text = f.writelines(new_lines)


    return replaced_file



md_files = glob.glob(os.path.join("yuque", "**/*.md"), recursive=True)
md_files.sort()

output_file = 'SUMMARY.md'

# 自动分组
groups = {}
chapters = md_files

for chapter in chapters:
    prefix = chapter.split('/')[1].split()[0][0]
    if prefix not in groups:
        groups[prefix] = []
    groups[prefix].append(chapter)

# 输出到目标文件
with open(output_file, 'w', encoding='utf-8') as f:
    f.write("# 目录\n\n")
    for group in sorted(groups.keys()):
        f.write(f"## {group}\n\n")
        for chapter in groups[group]:
            chapter_title = os.path.splitext(os.path.basename(chapter))[0]
            chapter = chapter.replace("yuque","pages")
            chapter = chapter.split()[0] + ".md"
            f.write(f"- [{chapter_title}]({chapter})\n")
        f.write("\n")
        

# 指定图片保存的目录路径
output_directory = "assets"

# 创建保存图片的目录
os.makedirs(output_directory, exist_ok=True)

for md_file in md_files[:]:
    print(md_file)
    replaced_file = localize_images(md_file, output_directory)
    #print(f"{md_file} -> {replaced_file}")